In [1]:
import pandas as pd

# Load the original CSV file
df = pd.read_csv("/DATA/rohan_kirti/niladri/dataset3/conversation.csv")  # replace with actual path if not in current directory

# Prepare variables
output_rows = []
agent_responses = []

# Temporary variables to track user query and agent reply
current_user_query = None
current_conversation_id = None
record_id = 1

for index, row in df.iterrows():
    if row['speaker'] == 'User':
        if current_user_query is not None:
            # Save previous user query without agent reply
            output_rows.append({
                'id': record_id,
                'conversation_id': current_conversation_id,
                'user_query': current_user_query,
                'agent_reply': ''
            })
            record_id += 1
        # Start new user query
        current_user_query = row['utterance']
        current_conversation_id = row['conversation_id']
    elif row['speaker'] == 'Agent':
        
        if current_user_query is not None:
            agent_responses.append(row['id'])
            # Save user query and agent reply
            output_rows.append({
                'id': record_id,
                'conversation_id': current_conversation_id,
                'user_query': current_user_query,
                'agent_reply': row['utterance']
            })
            record_id += 1
            current_user_query = None  # Reset after pairing

# If last entry is a user query without an agent reply
if current_user_query is not None:
    output_rows.append({
        'id': record_id,
        'conversation_id': current_conversation_id,
        'user_query': current_user_query,
        'agent_reply': ''
    })

# Convert to DataFrame
output_df = pd.DataFrame(output_rows)

# Save to CSV
output_df.to_csv("/DATA/rohan_kirti/niladri/dataset3/single/table_single.csv", index=False)

# Print or return list of agent responses
print(agent_responses)


[2, 4, 6, 8, 10, 13, 15, 17, 19, 21, 23, 27, 29, 31, 33, 35, 38, 40, 42, 44, 46]


In [2]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/DATA/rohan_kirti/niladri/dataset3/single/table_single.csv')

# Drop rows where 'agent_reply' is NaN or an empty string
df_cleaned = df[df['agent_reply'].notna() & (df['agent_reply'].str.strip() != '')]

# Save the cleaned DataFrame to a new CSV (or overwrite the original)
df_cleaned.to_csv('/DATA/rohan_kirti/niladri/dataset3/single/table_single.csv', index=False)


In [3]:
import pandas as pd
import json

df=pd.read_csv("/DATA/rohan_kirti/niladri/dataset3/single/table_single.csv")  # replace with actual path if not in current directory
# Output list for matching answers
matching_answers = []

# Read and process the JSONL file
with open('/DATA/rohan_kirti/niladri/dataset3/single/cleaned_output.jsonl', 'r') as f:
    for line in f:
        data = json.loads(line)
        if data.get('id_json') in agent_responses:
            matching_answers.append(data.get('answer'))

df["single agent"]= matching_answers
df.to_csv("/DATA/rohan_kirti/niladri/dataset3/single/table_single.csv", index=False)

In [4]:
import sys
sys.path.append('/DATA/rohan_kirti/.local/lib/python3.8/site-packages')

In [5]:
from rouge_score import rouge_scorer


In [6]:
import pandas as pd
from rouge_score import rouge_scorer

# Load CSV
df = pd.read_csv("/DATA/rohan_kirti/niladri/dataset3/single/table_single.csv")  # Replace with your actual file path

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

# Compute ROUGE scores
def compute_rouge(row):
    scores = scorer.score(row['single agent'], row['agent_reply'])
    return scores['rouge1'].fmeasure  # You can also use .precision or .recall

# Apply ROUGE calculation
df['rouge'] = df.apply(compute_rouge, axis=1)

# Save the updated DataFrame
df.to_csv("/DATA/rohan_kirti/niladri/dataset3/single/table_single.csv", index=False)